In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from datetime import datetime
import ast

In [ ]:
"""
index_uuid_df = pd.read_csv("/content/drive/MyDrive/metavis/index_uuid_df.csv")
similarities = pd.read_csv("/content/drive/MyDrive/metavis/similarities.csv")
similarities = similarities.iloc[: , 1:]
tag_common_matrix_array = pd.read_csv("/content/drive/MyDrive/metavis/tag_common_matrix_array.csv")
tag_common_matrix_array = tag_common_matrix_array.iloc[: , 1:]
lda_df_filter =  pd.read_csv("/content/drive/MyDrive/metavis/lda_df_filter.csv")
embedding_similarities = pd.read_csv("/content/drive/MyDrive/metavis/embedding_similaritie.csv")
embedding_similarities = embedding_similarities.iloc[: , 1:]
similarities = similarities.to_numpy()
embedding_similarities = embedding_similarities.to_numpy()
tag_common_matrix_array = tag_common_matrix_array.to_numpy()
"""

In [3]:
import pandas_gbq
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/metavis/teak-component-335117-5c025bc84ff5.json')

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "teak-component-335117" #Your-Project"

Get df with BigQuery

In [4]:
SQL_lda = """ SELECT * FROM `teak-component-335117.final_project_tables.lda_df_filter` """
lda_df = pandas_gbq.read_gbq(SQL_lda)

SQL_uuid = """ SELECT * FROM `teak-component-335117.final_project_tables.index_uuid_df` """
index_uuid_df = pandas_gbq.read_gbq(SQL_uuid)
index_uuid_df = index_uuid_df.sort_values(by=['index'],inplace = False)

Downloading: 100%|██████████| 3405/3405 [00:01<00:00, 3256.13rows/s]


Read similarities files from GCP bucket

In [5]:
from io import BytesIO, StringIO
from google.cloud import storage
from google.oauth2 import service_account

def get_byte_fileobj(project: str,
      bucket: str,
      path: str,
      service_account_credentials_path: str = None) -> BytesIO:

  blob = _get_blob(bucket, path, project, service_account_credentials_path)
  byte_stream = BytesIO()
  blob.download_to_file(byte_stream)
  byte_stream.seek(0)
  return byte_stream

def get_bytestring(project: str,
      bucket: str,
      path: str,
      service_account_credentials_path: str = None) -> bytes:

  blob = _get_blob(bucket, path, project, service_account_credentials_path)
  s = blob.download_as_string()
  return s

def _get_blob(bucket_name, path, project, service_account_credentials_path):
  if service_account_credentials_path:
      credentials = service_account.Credentials.from_service_account_file(service_account_credentials_path) 
  else:
    None
  storage_client = storage.Client(project = project, credentials = credentials)
  bucket = storage_client.get_bucket(bucket_name)
  blob = bucket.blob(path)
  return blob

In [6]:
fileobj_similarities = get_byte_fileobj('My First Poject', 'final_project_metavis', 'similarities.csv', '/content/drive/MyDrive/metavis/teak-component-335117-5c025bc84ff5.json')
similarities_matrix = pd.read_csv(fileobj_similarities).iloc[: , 1:].to_numpy()

In [7]:
fileobj_embedding_similarities = get_byte_fileobj('My First Poject', 'final_project_metavis', 'embedding_similaritie.csv', '/content/drive/MyDrive/metavis/teak-component-335117-5c025bc84ff5.json')
embedding_similarities = pd.read_csv(fileobj_embedding_similarities).iloc[: , 1:].to_numpy()

In [8]:
fileobj_tag_distance_matrix = get_byte_fileobj('My First Poject', 'final_project_metavis', 'tag_common_matrix_array.csv', '/content/drive/MyDrive/metavis/teak-component-335117-5c025bc84ff5.json')
tag_distance_matrix = pd.read_csv(fileobj_tag_distance_matrix).iloc[: , 1:].to_numpy()

In [13]:
index_uuid_with_info_raw = get_byte_fileobj('My First Poject', 'final_project_metavis', 'index_uuid_with_info.csv', '/content/drive/MyDrive/metavis/teak-component-335117-5c025bc84ff5.json')
index_uuid_with_info = pd.read_csv(index_uuid_with_info_raw).iloc[: , 1:]

In [ ]:
index_uuid_with_info

,index,data_uuid,name,attribution,viewCount,downloadCount,description,updateFrequency,rowsUpdatedAtDate
0,0,2232-dj5q,Local Law 37 - DYCD Report,"Department of Youth and Community Development,...",924,1057,Data on the youth who are currently residing i...,Monthly,2021-09-22 13:07:15
1,1,22gm-5ceg,Shapefiles of BOA and Community Brownfield Pla...,Mayor's Office of Environmental Remediation (M...,201,0,GIS shapefiles showing the areas in NYC where ...,Annually,2020-11-23 21:15:43
2,2,22rr-ujq3,2016-2017 Student Discipline Annual Report - ELL,NYC Department of Education,197,287,Student discipline annual report for English l...,Historical Data,2018-06-07 22:09:08
3,3,23z9-6uk9,2020 DOE High School Directory,Department of Education (DOE),4443,1094,Directory of Department of Education High Scho...,Historical Data,2019-10-23 22:15:58
4,4,242c-ru4i,DSNY Special Waste Drop-off Sites,Department of Sanitation,388,596,Location of DSNY Special Waste Drop-Off Sites....,Monthly,2021-11-08 11:09:27
...,...,...,...,...,...,...,...,...,...
3400,3400,zmut-au2w,DYCD after-school programs: Immigrant Services,Department of Youth and Community Development ...,1563,3220,"Facilities in New York City, by agency and sit...",As needed,2017-09-15 19:46:34
3401,3401,zpd4-gad8,2006 - 2011 NYS Math Test Results By Grade - B...,Department of Education (DOE),432,1284,New York City Results on the New York State Ma...,Historical Data,2011-10-05 22:14:41
3402,3402,zs4w-c9cd,2006 - 2011 English Language Arts (ELA) Test R...,Department of Education (DOE),216,1037,New York City Results on the New York State En...,Historical Data,2011-10-05 21:04:36
3403,3403,zt9s-n5aj,SAT (College Board) 2010 School Level Results,Department of Education (DOE),27073,43073,New York City school level College Board SAT r...,Historical Data,2012-02-29 15:36:56


### Function

In [67]:
def lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df,  embedding_similarities, num_recommendation = 5, 
                               spatial_weight=0.5, embedding_weight =0.4, semantic = True):
  rank_df = index_uuid_df.drop(index_uuid_df.columns.difference(['data_uuid']), 1)
  rank_df['name'] = index_uuid_df['name']

  print('Input dataset index: ', dataset_index, '\nName: ',list(index_uuid_df[index_uuid_df['index'] == dataset_index]['name'])[0])
  rank_df['spatial_similarity'] = similarities_matrix[dataset_index]
  rank_df['spatial_rank'] = rank_df['spatial_similarity'].rank(method='min', ascending=False)
  # Tag
  rank_df['tag_common'] =  tag_distance_matrix[dataset_index]
  rank_df['tag_rank'] = rank_df['tag_common'].rank(method='min', ascending=False) #rank_df['tag_distance'].rank(method='min', ascending=True)
  # Embedding
  rank_df['embedding_similarity'] = embedding_similarities[dataset_index]
  rank_df['embedding_rank'] = rank_df['embedding_similarity'].rank(method='min', ascending=False)
  # Weighted rank
  rank_df['weighted_rank'] = (1-spatial_weight-embedding_weight)*rank_df['tag_rank'] + spatial_weight*rank_df['spatial_rank'] + embedding_weight*rank_df['embedding_rank']
  rank_df_sorted = rank_df.sort_values(by=['weighted_rank'],inplace = False)
  rank_df_sorted = rank_df_sorted.reset_index(drop= True)
  rank_df_sorted_merge = rank_df_sorted.merge(lda_df, how='inner', on='data_uuid')
  
  if semantic:
    data_topic = rank_df_sorted_merge['topic'][0]
    rank_df_sorted_merge = pd.DataFrame(rank_df_sorted_merge[rank_df_sorted_merge['topic'] == data_topic])

  print('\nRecomendations:')
  for i in range(1,num_recommendation+1):
    print(list(rank_df_sorted_merge['name'])[i])

  return rank_df_sorted_merge

### Recommendations...

In [68]:
recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, 199, lda_df, index_uuid_df, embedding_similarities,
                                            num_recommendation = 5, spatial_weight=0.2, embedding_weight =0.4, semantic = False)
recommendation.head(6)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
Housing Database by City Council (Map)
Housing Database by Community District (Map)
Pedestrian Ramp Locations (Map)


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,topic
0,3zy2-a8eg,Street and Highway Capital Reconstruction Projects - Intersection (Map),1.000000,1.0,6,1.0,1.000000,1.0,1.0,0
1,si9g-fztb,Street and Highway Capital Reconstruction Projects - Block (Map),1.000000,1.0,6,1.0,0.980236,2.0,1.4,0
2,6q9w-6svb,Open Street Locations (Map),0.988428,10.0,1,8.0,0.823009,14.0,10.8,0
3,gf2p-d6pi,Housing Database by City Council (Map),0.976460,21.0,1,8.0,0.797212,30.0,19.4,8
4,m8qf-ycd5,Housing Database by Community District (Map),0.976460,21.0,1,8.0,0.778834,46.0,25.8,8
5,u7ws-2dus,Pedestrian Ramp Locations (Map),0.989071,8.0,1,8.0,0.764915,70.0,32.8,2


In [93]:
recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, 199, lda_df, index_uuid_df, embedding_similarities, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)
recommendation.head(6)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
New York City Truck Routes - Map
Street Pothole Work Orders - Closed (Map)
Outdoor Learning - Streets Locations Map (Historical)


,data_uuid,name,spatial_similarity,spatial_rank,tag_common,tag_rank,embedding_similarity,embedding_rank,weighted_rank,topic
0,3zy2-a8eg,Street and Highway Capital Reconstruction Projects - Intersection (Map),1.000000,1.0,6,1.0,1.000000,1.0,1.0,0
1,si9g-fztb,Street and Highway Capital Reconstruction Projects - Block (Map),1.000000,1.0,6,1.0,0.980236,2.0,1.4,0
2,6q9w-6svb,Open Street Locations (Map),0.988428,10.0,1,8.0,0.823009,14.0,10.8,0
7,wnu3-egq7,New York City Truck Routes - Map,0.988722,9.0,0,89.0,0.837958,5.0,39.4,0
8,emex-8upu,Street Pothole Work Orders - Closed (Map),0.998413,4.0,0,89.0,0.818021,21.0,44.8,0
11,u3t3-74n4,Outdoor Learning - Streets Locations Map (Historical),0.999441,3.0,1,8.0,0.748615,139.0,59.4,0


In [70]:
def connection(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df, embedding_similarities,
               num_recommendation = 5, spatial_weight=0.5, embedding_weight =0.4, semantic = True):
  
    base_recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, dataset_index, lda_df, index_uuid_df, embedding_similarities, num_recommendation, spatial_weight, embedding_weight, semantic)
    base_recommendation_uuid = base_recommendation['data_uuid'][:10].to_list()
    
    target_lst =  base_recommendation['data_uuid'][:10].to_list()
    dataset_uuid = index_uuid_df[index_uuid_df['index'] == dataset_index]['data_uuid'].to_list()[0]
    source_lst = [dataset_uuid]*10

    for i in base_recommendation_uuid:
      new_dataset_index = index_uuid_df[index_uuid_df['data_uuid'] == i]['index'].to_list()[0]
      recom = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, new_dataset_index, lda_df, index_uuid_df, embedding_similarities, num_recommendation, spatial_weight, embedding_weight, semantic)
      new_recom_uuid =  recom['data_uuid'][:10].to_list()
      for j in new_recom_uuid:
        if j in base_recommendation_uuid:
          source_lst.append(i)
          target_lst.append(j)
    print(source_lst)
    print(target_lst)
    connection_df = pd.DataFrame(zip(source_lst, target_lst), columns = ['source', 'target'])
    connection_df = connection_df[connection_df['source'] != connection_df['target']]
    return connection_df

 


In [90]:
connection(similarities_matrix, tag_distance_matrix, 199, lda_df, index_uuid_df, embedding_similarities, num_recommendation = 5, 
                                            spatial_weight=0.2, embedding_weight =0.4, semantic = True)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
New York City Truck Routes - Map
Street Pothole Work Orders - Closed (Map)
Outdoor Learning - Streets Locations Map (Historical)
Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Block (Map)
Open Street Locations (Map)
New York City Truck Routes - Map
Street Pothole Work Orders - Closed (Map)
Outdoor Learning - Streets Locations Map (Historical)
Input dataset index:  2662 
Name:  Street and Highway Capital Reconstruction Projects - Block (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Intersection (Map)
Open Street Locations (Map)
Street Pothole Work Orders - Closed (Map)
New York City Truck Routes - Map
DSNY Planned Comme

,source,target
1,3zy2-a8eg,si9g-fztb
2,3zy2-a8eg,6q9w-6svb
3,3zy2-a8eg,wnu3-egq7
4,3zy2-a8eg,emex-8upu
5,3zy2-a8eg,u3t3-74n4
6,3zy2-a8eg,a7bv-5698
7,3zy2-a8eg,vtjm-ngnu
8,3zy2-a8eg,5hsa-dfq5
9,3zy2-a8eg,as9z-kwsh
11,3zy2-a8eg,si9g-fztb


In [ ]:
# test
# 199, 333, 1001, 2002, 123

In [115]:
recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, 123, lda_df, index_uuid_with_info, embedding_similarities,
                                            num_recommendation = 5, spatial_weight=0.2, embedding_weight =0.4, semantic = False)
recommendation = recommendation.merge(index_uuid_with_info, how='inner', on='data_uuid')
recommendation['name'] = recommendation['name_x'] 
recommendation_show = recommendation[['name', 'description']]
#pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_colwidth', -1)
#recommendation_show.iloc[1:6, :]
recommendation_show.head(6)

Input dataset index:  123 
Name:  DYCD after-school programs:  Beacon Programs

Recomendations:
DYCD after-school programs: Jobs and Internships
DYCD after-school programs: Teen ACTION Programs
DYCD  after-school programs: Out Of School Time
DYCD after-school programs: OSY Out Of School Youth Employment Programs
DYCD after-school programs: Beacon Satellite At NYCHA Programs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


,name,description
0,DYCD after-school programs: Beacon Programs,"Facilities in New York City, by agency and site, that offer ""Beacon Programs"" after-school programs.\r\nUpdate Schedule: Annually"
1,DYCD after-school programs: Jobs and Internships,"Facilities in New York City, by agency and site, that offer the following after-school job and internship programs: Summer Youth Employment, In-School Youth Employment (ISY), Out-of-School Youth Employment (OSY), Youth Employment, and Adult Employment Programs for children in age groups 14 to 24, 16 to 21, children in all grades, and adults."
2,DYCD after-school programs: Teen ACTION Programs,"Facilities in New York City, by agency and site, that offer “Teen ACTION Programs” after-school programs for children ages 13 to 21."
3,DYCD after-school programs: Out Of School Time,"Facilities in New York City that offer ""Out of School Time"" after-school programs. Managed by the Department of Youth and Community Development (DYCD).\r\nUpdate Frequency: Annually"
4,DYCD after-school programs: OSY Out Of School Youth Employment Programs,"Facilities in New York City, by agency and site, that offer “Out-Of -School Youth Employment (OSY) Program” after-school job and internship programs for young adults ages 16 to 21."
5,DYCD after-school programs: Beacon Satellite At NYCHA Programs,"Facilities in New York City, by agency and site, that offer “Beacon Satellite at NYCHA Programs” after-school programs."


In [116]:
recommendation = lda_spatial_recommendation(similarities_matrix, tag_distance_matrix, 199, lda_df, index_uuid_with_info, embedding_similarities,
                                            num_recommendation = 5, spatial_weight=0, embedding_weight =0, semantic = False)
recommendation = recommendation.merge(index_uuid_with_info, how='inner', on='data_uuid')
recommendation['name'] = recommendation['name_x'] 
recommendation_show = recommendation[['name', 'description','tag_common']]
pd.set_option('display.max_colwidth', -1)
#recommendation_show.iloc[1:6, :]
recommendation_show.head(6)

Input dataset index:  199 
Name:  Street and Highway Capital Reconstruction Projects - Intersection (Map)

Recomendations:
Street and Highway Capital Reconstruction Projects - Intersection (Map)
Street and Highway Capital Reconstruction Projects - Block
Street and Highway Capital Reconstruction Projects - Block (Map)
Street Closures due to construction activities by Block
Capital Projects


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,name,description,tag_common
0,Street and Highway Capital Reconstruction Projects - Intersection,"This data is a spatial representation of street construction projects. Street and Highway capital projects are major street reconstruction projects, ranging from general street resurfacing projects to full reconstruction of the roadbed, sidewalks, sewer and water pipes and other utilities. Capital projects are essential to keep the City's infrastructure in a state of good repair.",6
1,Street and Highway Capital Reconstruction Projects - Intersection (Map),"This data is a spatial representation of street construction projects. Street and Highway capital projects are major street reconstruction projects, ranging from general street resurfacing projects to full reconstruction of the roadbed, sidewalks, sewer and water pipes and other utilities. Capital projects are essential to keep the City's infrastructure in a state of good repair.",6
2,Street and Highway Capital Reconstruction Projects - Block,"This data is a spatial representation of street construction projects. Street and Highway capital projects are major street reconstruction projects, ranging from general street resurfacing projects to full reconstruction of the roadbed, sidewalks, sewer and water pipes and other utilities. Capital projects are essential to keep the City's infrastructure in a state of good repair.",6
3,Street and Highway Capital Reconstruction Projects - Block (Map),"This data is a spatial representation of street construction projects. Street and Highway capital projects are major street reconstruction projects, ranging from general street resurfacing projects to full reconstruction of the roadbed, sidewalks, sewer and water pipes and other utilities. Capital projects are essential to keep the City's infrastructure in a state of good repair.",6
4,Street Closures due to construction activities by Block,"DOT Street Closure data identifies locations in the New York City Street Closure map where a street is subject to a full closure, restricting through traffic, for the purpose of conducting construction related activity on a City street. Details about DOT construction permits can be found at Street Works Manual, http://streetworksmanual.nyc/. Full Closure Permits are issued for a period of time during which the street may be closed to through traffic for only a portion of the time, and open at other times.",2
5,Capital Projects,"All major infrastructure and information technology projects with a budget of $25 million or more that are currently active (in the design, procurement, or construction phase).",2
